In [1]:
"""
使用说明：

功能概述：
- 用于浏览和选择图片文件，可将选中的图片复制到指定目录
- 自动保存浏览进度，下次打开时可继续上次的位置
- 每页显示15张图片（3行5列）

操作方法：
1. 浏览图片：
   - 点击"上一页"/"下一页"按钮
   - 使用键盘方向键 ←/→ 或 ↑/↓
   - 使用键盘 A/D 键

2. 选择图片：
   - 点击图片可选中/取消选中
   - 选中的图片会显示蓝色边框
   - 按 Q/E 键可全选/取消全选当前页的图片
   - 按数字键 1-5 可选择对应列的所有图片

3. 确认选择：
   - 点击"确认选择"按钮将所选图片复制到目标文件夹
   - 重复的图片会自动跳过

快捷键总览：
- ←/↑/A：上一页
- →/↓/D：下一页
- Q/E：全选/取消全选当前页
- 1-5：选择对应列的所有图片

文件说明：
- 源图片目录：test_img/
- 已选图片保存目录：selected_images/
- 页面和时间记录：config.json

注意事项：
- 支持的图片格式：jpg, jpeg, png, gif, bmp
- 会自动递归扫描源目录下的所有子文件夹
- 重启程序会自动加载上次浏览的页码
"""

'\n使用说明：\n\n功能概述：\n- 用于浏览和选择图片文件，可将选中的图片复制到指定目录\n- 自动保存浏览进度，下次打开时可继续上次的位置\n- 每页显示15张图片（3行5列）\n\n操作方法：\n1. 浏览图片：\n   - 点击"上一页"/"下一页"按钮\n   - 使用键盘方向键 ←/→ 或 ↑/↓\n   - 使用键盘 A/D 键\n\n2. 选择图片：\n   - 点击图片可选中/取消选中\n   - 选中的图片会显示蓝色边框\n   - 按 Q/E 键可全选/取消全选当前页的图片\n   - 按数字键 1-5 可选择对应列的所有图片\n\n3. 确认选择：\n   - 点击"确认选择"按钮将所选图片复制到目标文件夹\n   - 重复的图片会自动跳过\n\n快捷键总览：\n- ←/↑/A：上一页\n- →/↓/D：下一页\n- Q/E：全选/取消全选当前页\n- 1-5：选择对应列的所有图片\n\n文件说明：\n- 源图片目录：test_img/\n- 已选图片保存目录：selected_images/\n- 页面和时间记录：config.json\n\n注意事项：\n- 支持的图片格式：jpg, jpeg, png, gif, bmp\n- 会自动递归扫描源目录下的所有子文件夹\n- 重启程序会自动加载上次浏览的页码\n'

In [4]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import os
import shutil
from pathlib import Path
import glob
import time  # 添加到文件顶部的imports中
import re
import json  # 添加到文件顶部
from tkinter import messagebox  # 添加到文件顶部的imports中

class ImageSelector:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotation")
        
        # 设置窗口大小
        self.root.geometry("1920x1080")
        self.root.state('zoomed')  # 在 Windows 上最大化窗口
        self.root.focus_force()  # 聚焦窗口
        
        # 如果在其他操作系统上，可以使用：
        # self.root.attributes('-zoomed', True)  # Linux
        # self.root.attributes('-fullscreen', True)  # macOS
        
        
        # 初始化变量
        self.source_dir = Path("test_img")  # 源目录
        self.target_dir = Path("selected_images")  # 目标目录
        self.image_files = []  # 所有图片文件路径
        self.grid_rows = 3  # 行数
        self.grid_cols = 5  # 列数
        self.images_per_page = self.grid_rows * self.grid_cols
        self.image_padding = 10  # 图片间距
        self.min_image_size = 100  # 最小图片尺寸
        self.selected_images = set()  # 被选中的图片
        self.copied_images_count = self.count_existing_images()  # 添加这行来统计已复制的图片数量
        self.progress_bar = None  # 添加进度条变量
        
        # 添加状态栏变量
        self.status_label = None
        
        # 确保目标目录存在
        self.target_dir.mkdir(exist_ok=True)
        
        # 加载配置
        self.config_file = Path("config.json")  # 添加配置文件路径
        self.load_config()  

        
        # 添加时间跟踪变量
        self.session_start_time = time.time()  # 当前会话开始时间
        self.start_time = time.time()  # 程序启动时间
        self.page_start_time = time.time()  # 当前页面开始时间
        self.total_page_time = 0  # 总浏览时间（秒）
        self.page_times = []  # 记录每页浏览时间，用于计算平均值
        
        
        self.setup_ui()
        self.scan_images()
        self.display_current_page()
        
    def setup_ui(self):
        # 创建主框架
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # 创建图片显示区域框架，设置固定高度
        self.images_container = ttk.Frame(self.main_frame)
        self.images_container.pack(fill=tk.BOTH, expand=True)
        
        # 创建图片网格框架，设置固定高度
        self.images_frame = ttk.Frame(self.images_container, height=600)  # 设置固定高度
        self.images_frame.pack(fill=tk.BOTH, expand=True)
        self.images_frame.pack_propagate(False)  # 防止子组件影响框架大小
        
        # 创建图片标签网格
        self.image_labels = []
        for i in range(self.grid_rows):
            for j in range(self.grid_cols):
                frame = ttk.Frame(self.images_frame, borderwidth=3, relief="solid")
                frame.grid(row=i, column=j, padx=5, pady=5, sticky="nsew")
                
                label = ttk.Label(frame)
                label.pack(expand=True, fill=tk.BOTH, padx=5, pady=5)
                
                idx = i * self.grid_cols + j
                frame.bind("<Button-1>", lambda e, idx=idx: self.toggle_selection(idx))
                label.bind("<Button-1>", lambda e, idx=idx: self.toggle_selection(idx))
                
                self.image_labels.append(label)
        
        # 调整列权重
        for i in range(self.grid_cols):
            self.images_frame.grid_columnconfigure(i, weight=1)
        for i in range(self.grid_rows):
            self.images_frame.grid_rowconfigure(i, weight=1)
        
        # 控制按钮和状态栏区域
        self.controls_frame = ttk.Frame(self.main_frame)
        self.controls_frame.pack(fill=tk.X, pady=(10, 0))  # 增加顶部间距
        
        # 左侧按钮区域
        self.left_buttons_frame = ttk.Frame(self.controls_frame)
        self.left_buttons_frame.pack(side=tk.LEFT)
        
        self.prev_btn = ttk.Button(self.left_buttons_frame, text="上一页 (←/↑/A)", command=self.prev_page)
        self.prev_btn.pack(side=tk.LEFT, padx=5)
        
        self.next_btn = ttk.Button(self.left_buttons_frame, text="下一页 (→/↓/D)", command=self.next_page)
        self.next_btn.pack(side=tk.LEFT, padx=5)
        
        self.page_label = ttk.Label(self.left_buttons_frame, text="")
        self.page_label.pack(side=tk.LEFT, padx=20)
        
        # 右侧按钮区域
        self.right_buttons_frame = ttk.Frame(self.controls_frame)
        self.right_buttons_frame.pack(side=tk.RIGHT)
        
        self.confirm_btn = ttk.Button(self.right_buttons_frame, text="确认选择", command=self.confirm_selection)
        self.confirm_btn.pack(side=tk.RIGHT, padx=5)
        
        # 状态栏
        self.status_frame = ttk.Frame(self.controls_frame)
        self.status_frame.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=20)
        
        self.status_label = ttk.Label(self.status_frame, text="", wraplength=800)
        self.status_label.pack(fill=tk.X)
        
        # 添加进度条框架
        self.progress_frame = ttk.Frame(self.main_frame)
        self.progress_frame.pack(fill=tk.X, pady=(5, 10))
        
        # 添加进度条
        self.progress_bar = ttk.Progressbar(
            self.progress_frame, 
            orient="horizontal",
            length=300,
            mode="determinate"
        )
        self.progress_bar.pack(fill=tk.X, padx=10)
        
        # 添加键盘事件绑定
        self.root.bind('<Left>', lambda e: self.prev_page())
        self.root.bind('<Right>', lambda e: self.next_page())
        self.root.bind('<Up>', lambda e: self.prev_page())
        self.root.bind('<Down>', lambda e: self.next_page())
        self.root.bind('a', lambda e: self.prev_page())
        self.root.bind('A', lambda e: self.prev_page())
        self.root.bind('d', lambda e: self.next_page())
        self.root.bind('D', lambda e: self.next_page())
        
        # 添加新的键盘绑定
        self.root.bind('q', lambda e: self.toggle_select_all_current_page())
        self.root.bind('Q', lambda e: self.toggle_select_all_current_page())    
        self.root.bind('e', lambda e: self.toggle_select_all_current_page())
        self.root.bind('E', lambda e: self.toggle_select_all_current_page())
        self.root.bind('1', lambda e: self.select_column(0))
        self.root.bind('2', lambda e: self.select_column(1))
        self.root.bind('3', lambda e: self.select_column(2))
        self.root.bind('4', lambda e: self.select_column(3))
        self.root.bind('5', lambda e: self.select_column(4))
        
        # 添加窗口大小变化事件绑定
        self.root.bind('<Configure>', self.on_window_resize)
        
    def scan_images(self):
        # 递归扫描所有子文件夹中的图片
        image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
        self.image_files = []
        
        # 使用列表推导式收集所有图片
        for ext in image_extensions:
            self.image_files.extend(glob.glob(str(self.source_dir / '**' / f'*{ext}'), recursive=True))
        
        # 对文件路径进行自然排序
        self.image_files.sort(key=self.natural_sort_key)
        
        print(f"找到 {len(self.image_files)} 张图片")
        
    def natural_sort_key(self, path):
        """实现自然排序的键函数"""
        # 将路径转换为文件名
        filename = os.path.basename(path)
        # 使用正则表达式提取 {id} 和 {index}
        match = re.match(r'(\d+)_(\d+)\.jpg', filename)
        if match:
            id_part = int(match.group(1))
            index_part = int(match.group(2))
            return (id_part, index_part)
        return (filename,)

    def calculate_image_size(self):
        """根据图片显示区域计算最适合的图片尺寸"""
        # 获取图片显示区域的尺寸
        width = self.images_frame.winfo_width()
        height = self.images_frame.winfo_height()
        
        # 计算每个图片的最大可用空间
        available_width = (width - (self.grid_cols + 1) * self.image_padding) // self.grid_cols
        available_height = (height - (self.grid_rows + 1) * self.image_padding) // self.grid_rows
        
        # 取最小值确保图片完全适应空间
        size = min(available_width, available_height)
        
        # 确保尺寸不小于最小值且不大于最大值
        size = max(min(size, 300), self.min_image_size)  # 添加最大尺寸限制
        
        return (size, size)

    def on_window_resize(self, event):
        """窗口大小改变时的处理函数"""
        # 确保是主窗口的改变事件
        if event.widget == self.root:
            # 添加小延迟以避免过于频繁刷新
            self.root.after(100, self.refresh_display)

    def refresh_display(self):
        """刷新当前页面的显示"""
        self.display_current_page()

    def load_and_resize_image(self, image_path, size=None):
        """加载并调整图片大小"""
        try:
            if size is None:
                size = self.calculate_image_size()
                
            img = Image.open(image_path)
            
            # 按比例缩放图片，保持宽高比
            img.thumbnail(size, Image.Resampling.LANCZOS)
            
            # 创建新的白色背景图像
            background = Image.new('RGB', size, 'white')
            
            # 居中显示图片
            img_width, img_height = img.size
            x = (size[0] - img_width) // 2
            y = (size[1] - img_height) // 2
            background.paste(img, (x, y))
            
            return ImageTk.PhotoImage(background)
        except Exception as e:
            print(f"无法加载图片 {image_path}: {e}")
            return None

    def display_current_page(self):
        # 在显示新页面前，记录上一页的用时
        if len(self.page_times) > 0:
            page_time = time.time() - self.page_start_time
            self.total_page_time += page_time
            self.page_times.append(page_time)
        
        # 重置当前页计时器
        self.page_start_time = time.time()
        
        start_idx = self.current_page * self.images_per_page
        end_idx = start_idx + self.images_per_page
        current_files = self.image_files[start_idx:end_idx]
        
        # 计算当前图片尺寸
        current_size = self.calculate_image_size()
        
        # 清除前显示的所有图片
        for label in self.image_labels:
            label.configure(image="")
            label.image = None
            label.master.configure(style="")
        
        # 显示当前页的图片
        for i, image_path in enumerate(current_files):
            if i < len(self.image_labels):
                photo = self.load_and_resize_image(image_path, current_size)
                if photo:
                    self.image_labels[i].configure(image=photo)
                    self.image_labels[i].image = photo
                    
                    if image_path in self.selected_images:
                        self.image_labels[i].master.configure(style="Selected.TFrame")
        
        # 更新页码显示
        total_pages = (len(self.image_files) - 1) // self.images_per_page + 1
        self.page_label.configure(text=f"第 {self.current_page + 1} 页，共 {total_pages} 页")
        
        # 在显示页面后更新状态栏
        self.update_status_bar()
        
    def toggle_selection(self, idx):
        current_idx = self.current_page * self.images_per_page + idx
        if current_idx < len(self.image_files):
            image_path = self.image_files[current_idx]
            
            if image_path in self.selected_images:
                self.selected_images.remove(image_path)
                self.image_labels[idx].master.configure(style="")
            else:
                self.selected_images.add(image_path)
                self.image_labels[idx].master.configure(style="Selected.TFrame")
                
    def prev_page(self):
        if self.current_page > 0:
            self.current_page -= 1
            self.display_current_page()
            
    def next_page(self):
        max_pages = (len(self.image_files) - 1) // self.images_per_page
        if self.current_page < max_pages:
            self.current_page += 1
            self.display_current_page()
            
    def confirm_selection(self):
        copied_count = 0
        skipped_count = 0
        
        # 复制选中的图片到目标目录
        for image_path in self.selected_images:
            filename = os.path.basename(image_path)
            target_path = self.target_dir / filename
            
            if target_path.exists():
                skipped_count += 1
                continue
            
            shutil.copy2(image_path, target_path)
            copied_count += 1
        
        # 更新已复制图片的计数
        self.copied_images_count += copied_count
        
        print(f"已复制 {copied_count} 张图片到 {self.target_dir}")
        if skipped_count > 0:
            print(f"跳过 {skipped_count} 张已存在的图片")
            
        self.selected_images.clear()
        self.display_current_page()

    def toggle_select_all_current_page(self):
        start_idx = self.current_page * self.images_per_page
        end_idx = start_idx + self.images_per_page
        current_files = self.image_files[start_idx:end_idx]
        
        # 检查当前页面是否所有图片都被选中
        all_selected = all(image_path in self.selected_images for image_path in current_files)
        
        for i, image_path in enumerate(current_files):
            if all_selected:
                if image_path in self.selected_images:
                    self.selected_images.remove(image_path)
                    self.image_labels[i].master.configure(style="")
            else:
                if image_path not in self.selected_images:
                    self.selected_images.add(image_path)
                    self.image_labels[i].master.configure(style="Selected.TFrame")
        
        self.update_status_bar()

    def select_column(self, col_num):
        start_idx = self.current_page * self.images_per_page
        for row in range(3):  # 3行
            idx = row * 5 + col_num  # 5列
            image_idx = start_idx + idx
            if image_idx < len(self.image_files):
                image_path = self.image_files[image_idx]
                self.selected_images.add(image_path)
                self.image_labels[idx].master.configure(style="Selected.TFrame")
        
        self.update_status_bar()

    def update_status_bar(self):
        total_images = len(self.image_files)
        current_selected = len(self.selected_images)
        total_selected = self.copied_images_count + current_selected
        
        # 计算已浏览的图片数量和进度
        viewed_images = (self.current_page + 1) * self.images_per_page
        viewed_images = min(viewed_images, total_images)
        remaining_images = total_images - viewed_images
        
        # 计算当前会话用时
        current_session_time = time.time() - self.session_start_time
        # 计��实际总用时（历史总用时 + 当前会话用时）
        actual_total_time = self.total_time + current_session_time
        
        # 更新进度条
        progress_percentage = (viewed_images / total_images * 100) if total_images > 0 else 0
        self.progress_bar["value"] = progress_percentage
        
        # 格式化状态文本，使用换行来优化显示
        status_text = (
            f"已选择: {current_selected} | "
            f"已保存: {self.copied_images_count} | "
            f"总选择: {total_selected} | "
            f"剩余: {remaining_images}\n"
            f"总数: {total_images} | "
            f"进度: {progress_percentage:.1f}% | "
            f"本次用时: {self.format_time(current_session_time)} | "
            f"总用时: {self.format_time(actual_total_time)}"
        )
        
        self.status_label.configure(text=status_text)

    def count_existing_images(self):
        """统计目标文件夹中已存在的图片数量"""
        if not self.target_dir.exists():
            return 0
        image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
        count = sum(1 for f in self.target_dir.glob('*') 
                   if f.is_file() and f.suffix.lower() in image_extensions)
        return count

    def format_time(self, seconds):
        """将秒数转换为人类可读的时间格式"""
        if seconds < 60:
            return f"{seconds:.1f}秒"
        elif seconds < 3600:
            minutes = seconds / 60
            return f"{minutes:.1f}分钟"
        else:
            hours = seconds / 3600
            return f"{hours:.1f}小时"

    def load_config(self):
        """从JSON文件加载配置和进度"""
        if not self.config_file.exists():
            # 如果没有配置文件，初始化默认值
            self.current_page = 0
            self.total_time = 0.0
            self.copied_images_count = self.count_existing_images()
            self.save_config()  # 保存默认配置
            return
        
        try:
            with open(self.config_file, 'r', encoding='utf-8') as f:
                config = json.load(f)
                self.current_page = config.get('current_page', 0)
                self.total_time = config.get('total_time', 0.0)
                self.copied_images_count = config.get('copied_images_count', self.count_existing_images())
        except Exception as e:
            print(f"加载配置失败: {e}")
            self.current_page = 0
            self.total_time = 0.0
            self.copied_images_count = self.count_existing_images()

    def save_config(self):
        """保存配置到JSON文件"""
        try:
            session_time = time.time() - self.session_start_time
            current_total = self.total_time + session_time
            
            config = {
                'current_page': self.current_page,
                'total_time': current_total,
                'copied_images_count': self.copied_images_count,
                'last_update': time.strftime('%Y-%m-%d %H:%M:%S')
            }
            
            with open(self.config_file, 'w', encoding='utf-8') as f:
                json.dump(config, f, ensure_ascii=False, indent=4)
        except Exception as e:
            print(f"保存配置失败: {e}")

    def __del__(self):
        """在对象销毁时保存配置"""
        self.save_config()

def main():
    root = tk.Tk()
    
    # 创建自定义样式
    style = ttk.Style()
    style.configure("Selected.TFrame", background="light blue")
    
    app = ImageSelector(root)
    
    # 添加窗口关闭事件处理
    def on_closing():
        if app.selected_images:  # 检查是否有未确认的选择
            if tk.messagebox.askyesno("确认", "还有未确认的选择，是否在关闭前保存？"):
                app.confirm_selection()
        app.save_config()
        root.destroy()
    
    root.protocol("WM_DELETE_WINDOW", on_closing)
    root.mainloop()

if __name__ == "__main__":
    main()

找到 88105 张图片
已复制 6 张图片到 selected_images
跳过 1 张已存在的图片
已复制 4 张图片到 selected_images
